In [53]:
import graphlab as gl
from datetime import datetime as dt
import graphlab.aggregate as agg

In [54]:
columns_telecom=['id', 'time_interval','country_code', 'sms_in', 'sms_out', 'call_in', 'call_out', 'int_activity']

columns_province = ['id', 'province', 'time', 'sq_to_prov', 'pr_to_sq']


In [55]:
for j in range(1,32):
    file_name_txt = "milano_telecom_dec/sms-call-internet-mi-2013-12-%d.txt" %j
    file_name_csv = "milano_telecom_dec/sms-call-internet-mi-2013-12-%d.csv" %j
    add_column_names(file_name_txt,file_name_csv)

In [1]:
def add_column_names(file_name_txt,file_name_csv):
    day_dec = pd.read_csv(file_name_txt, names=columns_province, header=None, delimiter='\t')
    day_dec = day_nov.fillna(0)
    day_dec.to_csv(file_name_csv)

I used an approach here to read each csv file as a graphlab frame one be one and then processing on it. In the next phase i used 
graphlab to read al csv files from a directory as one frame. The idea was to check the time complexity of reading each csv as a frame and reading all csv in one frame.
Though i used virtual machine for this project, so keeping its slowness in mind, the performance was still better when i read the whole data as one frame.

In [2]:
def find_busy_time():
    for i in range(1,31):
        file_name = "milano_telecom_nov/sms-call-internet-mi-2013-11-%d.csv" %i
        nov_busy = nov_day_con(file_name)
        if i==1:
            pop_time = nov_busy
        else:
            pop_time = pop_time.join(nov_busy, on='time_interval', how= 'inner')
        return pop_time

In [3]:
def nov_day_con(file_name):
    nov_day = gl.SFrame.read_csv(file_name)
    nov_day['time_interval'] = nov_day['time_interval'].apply(lambda t:dt.utcfromtimestamp(t/1e3).strftime('%H'))
    nov_busy_time = nov_day.groupby(key_columns = 'time_interval', operations= {'smsin':agg.SUM('sms_in'),
                                                                                'smsout': agg.SUM('sms_out'), 
                                                                                'callin': agg.SUM('call_in'),
                                                                                'callout': agg.SUM('call_out'),
                                                                                'int_activity': agg.SUM('int_activity')})
    nov_busy_time['total'] = nov_busy_time['smsin']+ nov_busy_time['smsout'] +nov_busy_time['callin'] +nov_busy_time['callout'] +nov_busy_time['int_activity']
    nov_cong_time = nov_busy_time.sort('total', ascending=False)
    return nov_cong_time['time_interval', 'total'].head(10)

In [ ]:
busy_time_nov = find_busy_time()

In [ ]:
#Reading all data from directory as a single frame
dec = gl.SFrame.read_csv('milano_telecom_dec/')

In [4]:
def dec_day_con(dec):
    dec['time_interval'] = dec['time_interval'].apply(lambda t:dt.utcfromtimestamp(t/1e3).strftime('%H'))
    dec_busy_time = dec.groupby(key_columns = 'time_interval', operations= {'smsin':agg.SUM('sms_in'),
                                                                            'smsout': agg.SUM('sms_out'),
                                                                            'callin':agg.SUM('call_in'), 
                                                                            'callout': agg.SUM('call_out'),
                                                                            'int_activity': agg.SUM('int_activity')})
    dec_busy_time['total'] = dec_busy_time['smsin']+ dec_busy_time['smsout'] +dec_busy_time['callin'] +dec_busy_time['callout'] +dec_busy_time['int_activity']
    dec_cong_time = dec_busy_time.sort('total', ascending=False)
    return dec_cong_time['time_interval', 'total'].head(10)

In [ ]:
busy_time_dec = dec_day_con(dec)

In [5]:
def call_prov(file_name):
    pro = gl.SFrame.read_csv(file_name)
    famous_prov = pro.groupby(key_columns = 'province', operations= {'count': agg.COUNT()})
    famous_prov = famous_prov.sort('count', ascending=False)
    return famous_prov.head(6)

In [ ]:
most_called_prov = call_pro()

In [ ]:
social = gl.SFrame.read_csv("social_pulse_milan.csv", header=True)
languages_count = social.groupby(key_columns = 'language', operations= {'count': agg.COUNT()})
total_count = languages_count.sort('count', ascending=False)
total_count.print_rows(15,2)

In [6]:
#For Nov 26, 27, and 28 
def nov_day_comp(file_name):
    nov_day_milan = gl.SFrame.read_csv(file_name)
    nov_day_milan['time_interval'] = nov_day_milan['time_interval'].apply(lambda t:dt.utcfromtimestamp(t/1e3).strftime('%H'))
    nov_last_week = nov_day_milan.groupby(key_columns = 'time_interval', operations= {'callin':agg.SUM('call_in'), 
                                                                                      'callout': agg.SUM('call_out'), 
                                                                                      'int_activity': agg.SUM('int_activity')})
    nov_3_days = nov_last_week.sort('time_interval', ascending=True)
    return nov_3_days

In [7]:
def last_week_nov():
    for k in range(26,29):
        file_name = "milano_telecom_nov/sms-call-internet-mi-2013-11-%d.csv" %k
        nov_busy_week = nov_day_comp(file_name)
        if k==26:
            pop_week_nov = nov_busy_week
        else:
            pop_week_nov = pop_week_nov.join(nov_busy_week, on='time_interval', how= 'inner')
    return pop_week_nov

In [ ]:
nov_3day_com = last_week_nov()

In [8]:
#For Dec 24, 25, and 26
def last_week_dec():
    for k in range(24,27):
        file_name = "milano_telecom_nov/sms-call-internet-mi-2013-11-%d.csv" %k
        dec_busy_week = nov_day_comp(file_name)
        if k==26:
            pop_week_nov = dec_busy_week
        else:
            pop_week_nov = pop_week_nov.join(dec_busy_week, on='time_interval', how= 'inner')
    return pop_week_nov

In [ ]:
dec_3day_com = last_week_dec()

In [ ]:
compare_months = gl.SFrame()

In [ ]:
compare_months.show()

In [ ]:
compare_months['time'] = nov_3day_com['time_interval']
compare_months['nov_calls'] = nov_3day_com['callin'] + nov_3day_com['callout'] 
compare_months['nov_internet'] = nov_3day_com['int_activity']
compare_months['dec_calls'] = dec_3day_com['callin'] + dec_3day_com['callout'] 
compare_months['dec_internet'] = dec_3day_com['activity']

In [ ]:
torento_prov = gl.SFrame.read_csv('trento/trento_provinces')

In [9]:
def call_prov(file_name):
    famous_prov = pro.groupby(key_columns = 'province', operations= {'count': agg.COUNT()})
    famous_prov = famous_prov.sort('count', ascending=False)
    return famous_prov.head(5)

In [ ]:
call_prov(torento_prov)

In [ ]:
trento_teleco = gl.SFrame.read_csv('trento/trento_telecom/')

In [ ]:
def trento_busy_hours(dec):
    trento_telecom['time_interval'] = trento_telecom['time_interval'].apply(lambda t:dt.utcfromtimestamp(t/1e3).strftime('%H'))
    trento_telecom_busy = trento_telecom.groupby(key_columns = 'time_interval', operations= {'smsin':agg.SUM('sms_in'),
                                                                            'smsout': agg.SUM('sms_out'),
                                                                            'callin':agg.SUM('call_in'), 
                                                                            'callout': agg.SUM('call_out'),
                                                                            'int_activity': agg.SUM('int_activity')})
    trento_telecom_busy['total'] = trento_telecom_busy['smsin']+ dec_busy_time['smsout'] +dec_busy_time['callin'] +dec_busy_time['callout'] +dec_busy_time['int_activity']
    trento_cong_time = trento_telecom_busy.sort('total', ascending=False)
    return trento_cong_time['time_interval', 'total']

In [ ]:
trento_telecom = gl.SFrame.read_csv('trneto/trento_telecom')

In [ ]:
def dec_day_con(dec):
    dec_busy_time = dec.groupby(key_columns = 'time_interval', operations= {'smsin':agg.SUM('smsin'),
                                                                            'smsout': agg.SUM('smsout'), 
                                                                            'callin':agg.SUM('callin'), 
                                                                            'callout': agg.SUM('callout'),
                                                                            'int_activity': agg.SUM('int_activity')})
    dec_busy_time['total'] = dec_busy_time['smsin']+ dec_busy_time['smsout'] +dec_busy_time['callin'] +dec_busy_time['callout'] +dec_busy_time['int_activity']
    dec_cong_time = dec_busy_time.sort('total', ascending=False)
    return dec_cong_time['time_interval', 'total']

In [ ]:
trento_busy_hours = dec_day_con(trento_time)

In [ ]:
trento_time = gl.SFrame(trento_telecom)

In [ ]:
trento_plots.show(columns=['time_interval', 'total'],view='Heat Map', x='time_interval', y='total')

In [ ]:
weather = gl.SFrame.read_csv('milano_weather.csv')

In [ ]:
clear_weather = gl.SFrame()
rain_weather = gl.SFrame()
snow_weather = gl.SFrame()

In [ ]:
clear_weather = weather.filter_by(0, 'type')
rain_weather = weather.filter_by(1, 'type')
snow_weather = weather.filter_by(2, 'type')

In [ ]:
clear_weather = clear_weather.join(trento_usage, on='timestamp', how='inner')
rain_weather = rain_weather.join(trento_usage, on='timestamp', how='inner')
snow_weather = snow_weather.join(trento_usage, on='timestamp', how='inner')

In [ ]:
import seaborn as sns
sns.set()
sns.distplot(clear['total'], kde=False)
sns.distplot(rain['total'], kde=False)
sns.distplot(snow['total'], kde=False)

plt.legend(['Clear', 'Rain', 'Snow'])

plt.axvline(0, color="k", linestyle="--");
plt.show()